In [ ]:
%run network-init.ipynb

In [ ]:
import recordlinkage
import utils
import itertools

# Read some subset to match

In [ ]:
df = pd.read_pickle("dataframe.pickled")

In [ ]:
df = df[df.FonetiskNavn.str.startswith("TT")]

In [ ]:
df = df[df.Køn==False] # oh my god det er grimt, men sådan er pandas syntax

In [ ]:
grouped = df.groupby("FT")

In [ ]:
indexer = recordlinkage.indexing.SortedNeighbourhoodIndex(on="Fødeår", window=5)

In [ ]:
def generate_scores(d):
    lots = []
    for (a, b) in d:
        a = dfA.loc[a]
        b = dfB.loc[b]
        lots.append((a.FT, a.Kipnr, a.Løbenr, b.FT, b.Kipnr, b.Løbenr) + score(a,b))
    return lots

For looking up people by FT-kip-løb keys later...

In [ ]:
lookup = df.set_index(["FT", "Kipnr", "Løbenr"], inplace=False).sort_index(inplace=False)

In [ ]:
potential_matches = set() # no idea why duplicates occur
dfA = dfB = None
yearA = yearB = None

for year, data in grouped:
    yearB = year
    dfB = data
    if dfA is not None:
        print("Now try", yearA, "with", len(dfA), "vs", yearB, "with", len(dfB))
        index = indexer.index(dfA, dfB)
        print("Index of size", len(index))
        with multiprocessing.Pool() as p:
            res = p.map(generate_scores, utils.chunks(index, 500))
        df_eval = pd.DataFrame(list(itertools.chain(*res)),
                               columns=["a_FT", "a_Kipnr", "a_Løbenr",
                                        "b_FT", "b_Kipnr", "b_Løbenr"] + score_columns)
        del(res)
        input_fn = tf.estimator.inputs.pandas_input_fn(x=df_eval, shuffle=False)
        prediction = model1.predict(input_fn)
        for i, d in enumerate(prediction):
            if d["classes"][0] == b"1":
                r = df_eval.iloc[i]
                potential_matches.add(tuple(r[:6]) + (d["probabilities"][1],))
        print("Now have", len(potential_matches))
    dfA = dfB
    yearA = yearB

In [ ]:
with open("m-TT.csv", "w", encoding="utf-8") as fd:
    writer = csv.writer(fd, lineterminator="\n", delimiter="|")
    writer.writerow("a_FT a_Kipnr a_Løbenr b_FT b_Kipnr b_Løbenr p".split())
    writer.writerows(potential_matches)

In [ ]:
utils.datadir.stem

In [ ]:
header = "Amt|Herred|Sogn|Navn|Køn|Fødested|Fødeår|Civilstand|Position|Erhverv|Kipnr|Løbenr|Group".split("|")

In [ ]:
import textwrap

In [ ]:
pd.read_csv("../scripts/m-TT.csv", delimiter="|")

In [ ]:
for a_FT, a_Kip, a_Løb, b_FT, b_Kip, b_Løb, prob in potential_matches[:100]:
    print(prob)
    r = df_eval.iloc[i]
    a = lookup.loc[(a_FT, a_Kip, a_Løb)]
    b = lookup.loc[(b_FT, b_Kip, b_Løb)]
    for i in range(10):
        print("{:10s} {:40s} {:40s}".format(header[i],
                                            textwrap.shorten(str(a[i]),40), 
                                            textwrap.shorten(str(b[i]), 40)))
    print()

In [ ]:
for var in model1.get_variable_names():
    print(var)
    print(model1.get_variable_value(var))